In [86]:
import physionet_challenge_utility_script as pc

import ecg_plot
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import pad_sequences

print(1)

%load_ext autoreload
%autoreload
%reload_ext autoreload


1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
DATADIR = "./SM22_MLData"

In [88]:
firstModelData = {}
firstModelData["gender"], firstModelData["age"], firstModelData["labels"], firstModelData["ecg_filenames"] = pc.import_key_data(DATADIR)
firstModelData["ecg_filenames"] = np.asarray(firstModelData["ecg_filenames"])

In [89]:
cache = firstModelData["labels"]

In [90]:
"""
Remaps labels into class values (Integer Encoding)
val: totalRhythm
1 - SR
2 - NSR
"""

print(len(firstModelData["labels"]))
for i in range(len(firstModelData["labels"])):
    splitLabels = firstModelData["labels"][i].split(",")
    isSR = 0
    if "426783006" in splitLabels:
        isSR = 1 #SR
    firstModelData["labels"][i] = isSR


43101


In [91]:
firstModelData["labels"].count('1'), firstModelData["labels"].count('0')

(20846, 22255)

In [92]:
firstModelData["labels"][0] = "undefined"

In [93]:
firstModelData["age"], firstModelData["gender"] = pc.import_gender_and_age(firstModelData["age"], firstModelData["gender"])

gender data shape: 43101
age data shape: 43101


In [94]:
SNOMED_scored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_scored.csv", sep=";")
SNOMED_unscored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/FIRSTMODELMapping.csv", sep=",")

In [95]:
df_labels = pd.DataFrame(firstModelData["labels"])

In [128]:
y , snomed_classes = pc.onehot_encode(df_labels)

The classes we will look at are encoded as SNOMED CT codes:
['0' '1' 'undefined']
classes: 2


In [120]:
y_all_comb = pc.get_labels_for_all_combinations(y)

1

In [98]:
folds = pc.split_data(firstModelData["labels"], y_all_comb)

Training split: 38790
Validation split: 4311


/Users/ksv/miniforge3/envs/mlp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [99]:
order_array = folds[0][0]

In [100]:
def shuffle_batch_generator_demo(batch_size, gen_x,gen_y, gen_z): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    batch_demo_data = np.zeros((batch_size,2))
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            batch_demo_data[i] = next(gen_z)

        X_combined = [batch_features, batch_demo_data]
        yield X_combined, batch_labels
        
def shuffle_batch_generator(batch_size, gen_x,gen_y): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            
        yield batch_features, batch_labels

def generate_y_shuffle(y_train):
    while True:
        for i in order_array:
            y_shuffled = y_train[i]
            yield y_shuffled


def generate_X_shuffle(X_train):
    while True:
        for i in order_array:
                #if filepath.endswith(".mat"):
                    data, header_data = pc.load_challenge_data(X_train[i])
                    X_train_new = pad_sequences(data, maxlen=5000, truncating='post',padding="post")
                    X_train_new = X_train_new.reshape(5000,12)
                    yield X_train_new

def generate_z_shuffle(age_train, gender_train):
    while True:
        for i in order_array:
            gen_age = age_train[i]
            gen_gender = gender_train[i]
            z_train = [gen_age , gen_gender]
            yield z_train

In [101]:
new_weights=pc.calculating_class_weights(y)

In [102]:
keys = np.arange(0,3,1)
weight_dictionary = dict(zip(keys, new_weights.T[1]))
weight_dictionary

{0: 0.9683877055810192, 1: 1.0337954523649622}

In [103]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_AUC', factor=0.1, patience=2, verbose=1, mode='max',
    min_delta=0.0001, cooldown=0, min_lr=0
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=4)

In [104]:
from scipy import optimize
def thr_chall_metrics(thr, label, output_prob):
    return -pc.compute_challenge_metric_for_opt(label, np.array(output_prob>thr))

In [105]:
def myFCN(classes=27):
    inputlayer = keras.layers.Input(shape=(5000,12)) 

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8,input_shape=(5000,12), padding='same')(inputlayer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv1)

    outputlayer = keras.layers.Dense(classes, activation='sigmoid')(gap_layer)

    model = keras.Model(inputs=inputlayer, outputs=outputlayer)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

    return model

In [106]:
model = pc.residual_network_1d(classes=2)

In [107]:
#model.load_weights(f"modelweights_newshape.h5")

In [110]:
batchsize = 30
history = model.fit(
    x=shuffle_batch_generator(batch_size=batchsize, gen_x=generate_X_shuffle(firstModelData["ecg_filenames"]), 
    gen_y=generate_y_shuffle(y)), 
    epochs=5, 
    steps_per_epoch=(len(order_array)/batchsize), 
    validation_data=pc.generate_validation_data(firstModelData["ecg_filenames"],y,folds[0][1]), 
    validation_freq=1, 
    class_weight=weight_dictionary, 
    callbacks=[reduce_lr],
)

Epoch 1/5
1293/1293 [==============================] - ETA: 0s - loss: 0.5428 - accuracy: 0.7313 - Recall: 0.7313 - Precision: 0.7313 - AUC: 0.8030

2022-03-15 19:42:06.691363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1293/1293 [==============================] - 1862s 1s/step - loss: 0.5428 - accuracy: 0.7313 - Recall: 0.7313 - Precision: 0.7313 - AUC: 0.8030 - val_loss: 1.1192 - val_accuracy: 0.5350 - val_Recall: 0.5350 - val_Precision: 0.5349 - val_AUC: 0.7641 - lr: 0.0100
Epoch 2/5
1293/1293 [==============================] - 1837s 1s/step - loss: 0.4950 - accuracy: 0.7632 - Recall: 0.7632 - Precision: 0.7632 - AUC: 0.8422 - val_loss: 1.4975 - val_accuracy: 0.5397 - val_Recall: 0.5397 - val_Precision: 0.5396 - val_AUC: 0.7097 - lr: 0.0100
Epoch 3/5
 230/1293 [====>.........................] - ETA: 25:10 - loss: 0.4826 - accuracy: 0.7733 - Recall: 0.7733 - Precision: 0.7733 - AUC: 0.8534

KeyboardInterrupt: 

In [ ]:
model.save_weights("modelweights_binary1.h5")

In [111]:
model.load_weights("modelweights_newShape31.h5")

ValueError: Cannot assign value to variable ' dense_2/kernel:0': Shape mismatch.The variable shape (128, 2), and the assigned value shape (128, 3) are incompatible.